###**infant_hfb_ml**

This notebook address three goals, each linked to a primary result in Holubecki et al., 2025:  
1. Analyze high-frequency broadband activity (HFB: 70-150 Hz) using Python with MNE, reproducing results reported from analysis using MATLAB with FieldTrip. Successfully converted FieldTrip preprocessing pipeline to MNE, ensuring identical parameter matching (multitaper method, frequency bands, and normalization).
2. Classify wake vs. sleep states using all available HFB data from a single channel. Using logistic regression with 70/30 train/test split across all subjects, Fz HFB (located over the anterior fontanelle with minimal skull attenuation) successfully discriminated states (ROC-AUC = 0.786). By contrast, P3 HFB (over relatively thick skull) showed chance-level performance (ROC-AUC = 0.500), as expected given equivalent mean HFB values between states. The full sample includes data (20+ samples per state) from 17 subjects with Fz and 14 subjects with P3.
3. Classify wake vs. sleep states using limited training data (5 samples per state from 10 subjects). Using 10-fold cross-validation, Fz HFB showed modest discrimination ability with limited data, while P3 performed at chance level, consistent with the ground truth that no signal exists in P3. This demonstrates the feasibility of classification with minimal training data when a true signal is present.

Copyright (c) 2025  
EL Johnson, PhD

###Import modules:

In [ ]:
import gdown
import zipfile
import matplotlib.pyplot as plt

!pip install mne  # for custom modules

# custom modules
from hfb_utils import load_eeg_files, compute_hfb, create_hfb_df
from ml_utils import classify_wake_sleep, compare_classifiers, classify_kfold

###Download data from Google Drive:

In [ ]:
# download
fid = '1DX7D6iWITATB8Tn5oLuez4rJj3obWhOA'
url = f'https://drive.google.com/uc?id={fid}'
gdown.download(url, 'All_Data_clean_z.zip', quiet = False)

# unzip
with zipfile.ZipFile('All_Data_clean_z.zip', 'r') as zip_ref:
  zip_ref.extractall()

### Load the data

Each MATLAB input file contains two data structures per subject (`awake_eeg` and `sleep_eeg`), which are the output of FieldTrip's `ft_preprocessing`. Each structure contains the following variables of interest:
- `label` = channel names
- `time` = time vectors for each trial (sec)
- `trial` = EEG data (channels x timepoints per epoch)
- `fsample` = sampling rate (Hz)

The `load_eeg_files()` function converts these FieldTrip structures directly into MNE `Epochs` objects. Each output is a dictionary containing two MNE `Epochs` objects per subject (e.g., `Subj1_awake` and `Subj1_sleep`), preserving all channel info, trial data, and sampling parameters from the original MATLAB structures.

In [ ]:
all_data = load_eeg_files()

In [ ]:
all_data['Subj1_awake']  # take a look

### Goal 1: Reproduce MATLAB-FieldTrip HFB analysis using Python-MNE

**Conversion process**: The FieldTrip analysis computed PSD using the multitaper method (`ft_freqanalysis` with `method = 'mtmfft'`, `taper = 'dpss'`). The equivalent MNE function is `mne.time_frequency.psd_array_multitaper`. The challenge was matching FieldTrip's exact frequency bins and parameters to ensure identical results.  

**Key parameter matching**:
- **Frequency range**: FieldTrip used `cfg.foi = 75:10:145` (8 specific frequencies). MNE computes all frequencies in the range, requiring selection of the closest bins to match FieldTrip's exact frequencies.
- **Spectral smoothing**: FieldTrip's `cfg.tapsmofrq = 10/2` specifies half-bandwidth (+/-5 Hz smoothing). This maps to MNE's `bandwidth = 10` parameter.
- **Normalization**: Both FieldTrip's `output = 'pow'` and MNE's `normalization = 'full'` return PSD in units of power/Hz.
- **Frequency normalization**: Following the FieldTrip pipeline, PSD values were multiplied by their corresponding frequencies, then averaged across frequency bands.  

**Verification**: Comparing mean HFB power values from both pipelines confirmed success:
- Fz: awake mean = 0.0152, sleep mean = 0.0064 (awake > sleep, as expected)
- P3: awake mean = 0.0236, sleep mean = 0.0238 (awake essentially = sleep, as expected)

In [ ]:
all_hfb = compute_hfb(all_data)

In [ ]:
all_hfb['Subj1'].keys()  # take a look

###Select data for ML analysis:  

Each model uses data from a single channel:
- Fz, located over the anterior fontanelle. Hierarchical modeling revealed the greatest HFB wake > sleep difference in Fz. ML models with Fz HFB data as the feature should classify wake vs. sleep states.
- P3, located over relatively thick skull. Hierarchical modeling indicated a non-significant HFB wake-sleep difference in P3. ML models with P3 HFB data as the feature should not classify wake vs. sleep states.  

In [ ]:
df = create_hfb_df(all_hfb)

print(f'Created df with {len(df)} total epochs:')
print(f'  Awake: {(df.condition == "awake").sum()}')
print(f'  Sleep: {(df.condition == "sleep").sum()}')
print(f'  Fz total: {df.fz_hfb.notna().sum()}')
print(f'  P3 total: {df.p3_hfb.notna().sum()}')

Summarize Fz and P3 data to confirm:

In [ ]:
fig, ax = plt.subplots(1, 2, figsize = (8, 3))

for idx, channel in enumerate(['fz_hfb', 'p3_hfb']):
  df_clean = df[df[channel].notna()].copy()

  awake_vals = df_clean[df_clean.condition == 'awake'][channel]
  sleep_vals = df_clean[df_clean.condition == 'sleep'][channel]

  ax[idx].hist(sleep_vals, bins = 50, alpha = 0.5, label = 'sleep', color = 'dimgray')
  ax[idx].hist(awake_vals, bins = 50, alpha = 0.5, label = 'awake', color = 'darkred')
  ax[idx].set_xlabel('HFB power')
  ax[idx].set_ylabel('Count')
  ax[idx].set_title(f'{channel} distribution')
  ax[idx].legend()

  print(f'{channel}:')
  print(f'  Awake mean: {awake_vals.mean():.4f}, SD: {awake_vals.std():.4f}')
  print(f'  Sleep mean: {sleep_vals.mean():.4f}, SD: {sleep_vals.std():.4f}')

plt.tight_layout()
plt.show()

### Goal 2: Classify wake vs. sleep states using all available data

**Classification approach**: Logistic regression with a 70/30 train/test split classified wake vs. sleep states based on single-channel HFB power. The dataset was stratified to maintain equal class proportions across splits. Balanced class weights addressed the imbalance between awake (1135 epochs) and sleep (2078 epochs) samples, ensuring the model learned to predict both classes rather than defaulting to the majority class.

**Why logistic regression**: Four classifiers (logistic regression, SVM with RBF kernel, random forest, and gradient boosting) were compared to determine the optimal approach. Logistic regression was selected because its constraint to finding a simple linear threshold prevents overfitting to noise. Complex models (SVM, random forest, and gradient boosting) showed high performance for both Fz and P3, contradicting the ground truth that P3 contains no discriminative signal. These models overfit to random patterns in the training data. Logistic regression provided valid performance estimates: good for Fz (real signal) and chance-level for P3 (no signal).  

**Results**:
- **Fz HFB**: ROC-AUC = 0.786, balanced accuracy = 0.661, demonstrating moderate discrimination ability between wake and sleep states
- **P3 HFB**: ROC-AUC = 0.500, balanced accuracy = 0.500, demonstrating chance-level performance, as expected given no true signal

**Interpretation**: Fz HFB successfully discriminates wake from sleep states, consistent with higher power during wake compared to sleep. P3 HFB shows no discrimination ability, consistent with equivalent power across states. These results validate that skull thickness affects HFB signal quality, with minimal attenuation over the anterior fontanelle (Fz) and substantial attenuation over thick skull (P3).  

The random split approach (mixing epochs across all subjects in train and test sets) tests generalization to new samples but not to new subjects. When tested with a subject-based split, performance degraded substantially, indicating that the model learned population-level patterns rather than subject-invariant features. This suggests that for deployment in clinical or research settings, training data including the target subject would be necessary. The moderate ROC-AUC reflects both the overlap in HFB distributions between states and challenge of cross-subject generalization with a single feature.

In [ ]:
# run logistic regression classifier
fz_results = classify_wake_sleep(df, channel = 'fz_hfb')
p3_results = classify_wake_sleep(df, channel = 'p3_hfb')

In [ ]:
# compare classifiers
fz_comparison = compare_classifiers(df, channel = 'fz_hfb')
p3_comparison = compare_classifiers(df, channel = 'p3_hfb')

### Goal 3: Classify wake vs. sleep states using limited data

**Classification approach**: To test HFB discriminability with minimal data, logistic regression with 10-fold cross-validation classified wake vs. sleep states using only 5 samples per condition from 10 subjects (100 total samples). This approach simulates a realistic scenario where extensive data collection is impractical. Stratified 10-fold CV ensured balanced class representation across folds, with each fold training on 90 samples and testing on 10 samples. The same subjects were used for both Fz and P3 analyses to enable direct comparison.  

**Why 10-fold CV**: With only 100 samples, 10-fold CV provides a balance between training set size (90 samples per fold) and evaluation stability (10 folds for averaging). This approach tests the hypothesis from Holubecki et al., 2025 that HFB wake/sleep differences can be detected with high reliability using as few as 5 samples per state from as few as 10 subjects. The 10-fold CV framework assesses classification performance under these minimal data conditions while providing robust performance estimates by averaging across folds.  

**Results**:
- **Fz HFB**: mean ROC-AUC = 0.656, mean accuracy = 0.59, demonstrating above-chance discrimination with limited training data
- **P3 HFB**: mean ROC-AUC = 0.326, mean accuracy = 0.48, demonstrating chance-level performance as expected

**Interpretation**: Classification with limited training data remains feasible for Fz but not P3, consistent with the presence of a true discriminative signal in Fz. The performance decrease from Goal 2 (2249 training samples) reflects the challenge of learning from limited data, but the maintained above-chance performance demonstrates practical viability. This suggests that brief data collection sessions (~5 epochs per state, i.e., 25 sec per state) from a modest sample (10 subjects) can support wake/sleep classification using Fz HFB.

In [ ]:
# run 10-fold CV
fz_results_10fold = classify_kfold(df, channel = 'fz_hfb', n_subj = 10, n_per_cond = 5, n_folds = 10)
p3_results_10fold = classify_kfold(df, channel = 'p3_hfb', n_subj = 10, n_per_cond = 5, n_folds = 10)